<a href="https://www.kaggle.com/code/andreidolzhikov/baseline-v1?scriptVersionId=112968615" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

# DATA_DIR = 'data'
# df_train = pd.read_csv('data/hotels_train.csv') # датасет для обучения
# df_test = pd.read_csv('data/hotels_test.csv') # датасет для предсказания
# sample_submission = pd.read_csv('data/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

### Отсюда начнем преобразования

Работа с признаком адреса

In [ ]:
# функция выделения из текста города
def adress_analysis(text):
    if text.split()[-1] == 'Kingdom':
        city = text.split()[-5]
        country = ' '.join(text.split()[-2:])
    else:
        city = text.split()[-2]
        country = text.split()[-1]
    return city

# создаем признак города
data['city'] = data['hotel_address'].apply(adress_analysis)

# удаляем признак адреса
data = data.drop(columns=['hotel_address'])

Кодируем признак названия и удаляем его

In [ ]:
# для признака названия отеля производим двоичную кодировку
import category_encoders as ce # импорт для работы с кодировщиком
bin_encoder = ce.BinaryEncoder(cols=['hotel_name']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(data['hotel_name'])
data = pd.concat([data, type_bin], axis=1)

# удаляем признак названия отеля
data = data.drop(columns=['hotel_name'])

Кодируем признак национальности и удаляем его

In [ ]:
# убираем лишние пробелы из элементов признака
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x.strip().lower())
# создаем список из 10 наиболее часто встречающихся национальностей
most_common_nations = data['reviewer_nationality'].value_counts(normalize=True)[:10].index.tolist()
# оставляем только национальности из списка, остальные Other
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in most_common_nations else 'other')

# применяем однократное кодирование для признака (11 признаков)
import category_encoders as ce # импорт для работы с кодировщиком

encoder = ce.OneHotEncoder(cols=['reviewer_nationality']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

# удаляем исходный признак национальности
data = data.drop(columns=['reviewer_nationality'])

кодируем признак города

In [ ]:
encoder = ce.OneHotEncoder(cols=['city']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['city'])
data = pd.concat([data, type_bin], axis=1)

# удаляем исходный признак национальности
data = data.drop(columns=['city'])

Действия выше --- MAPE: 0.1387163072970709

Пропуски только в столбцах с широтой и долготой. Если в строке пропуск, удаляем строку

Работа с негативными отзывами

In [ ]:
data['negative_review'] = data['negative_review'].apply(lambda x: x.strip().lower())
# список взят из (https://www.kaggle.com/code/motoborgrus/proj-3-data/notebook#%D0%9F%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA-hotel_address)
no_negative_review = ['No Negative', 'Nothing', 'nothing', 'None', 'N A', '',
 'Nothing really', 'N a', 'All good', 'No complaints', 'Nothing at all', 'Nothing to dislike',
 'none', 'Nil', 'Everything was perfect', 'Can t think of anything', 'n a', 'Absolutely nothing',
 'Everything was great', 'Nothing to complain about', 'Nothing not to like', 'NA', 'I liked everything',
 'NOTHING', 'No', 'Everything was good', 'Liked everything', 'Not much', 'Nothing all good', 'Nothing in particular',
 'Everything was fine', 'Na', 'All was good', 'Nothing it was perfect', 'Non', 'Nothing comes to mind', 'There was nothing I didn t like',
 'No complaints at all', 'Nothing everything was perfect', 'We liked everything', 'Nothing I didn t like', 'No negatives', 'There was nothing to dislike',
 'Nothing everything was great', 'nothing really', 'Nothing to report', 'Not a thing', 'It was all good', 'There was nothing we didn t like',
 'Loved everything', 'No thing', 'no complaints', 'All ok', 'all good', 'There was nothing not to like', 'No issues', 'Nothing I can think of', 'No bad experience',
 'Nothing to say', 'No dislikes', 'Nothing to mention', 'no', 'Everything was excellent', 'Nothing we didn t like', 'No problems', 'Nothing to complain',
 'Nada', 'Nothing bad to say', 'everything', 'nil', 'nothing at all', 'I loved everything', 'Nothing bad', 'All great']

no_negative_review = [i.lower() for i in no_negative_review]
data['is_negative'] = data['negative_review'].apply(lambda x: 0 if x in no_negative_review else 1)
data = data.drop(columns=['negative_review'])

Работа с позитивными отзывами

In [ ]:
data['positive_review'] = data['positive_review'].apply(lambda x: x.strip().lower())
no_positive_review = ['no positive', 'nothing', 'n a', 'none', '', ' ' , 'not much', 'nothing at all']
data['is_positive'] = data['positive_review'].apply(lambda x: 0 if x in no_positive_review else 1)
data = data.drop(columns=['positive_review'])

Перевод признака даты в количество дней

In [ ]:
data['review_date'] = pd.to_datetime(data['review_date'])
data['days_from_rewiew'] = (data['review_date'].max() - data['review_date']).dt.days
# удаляем признак даты и признак days since rewiew, т.к. дублируется
data = data.drop(columns=['review_date'])
data = data.drop(columns=['days_since_review'])

Пропуски только в столбцах с широтой и долготой. Если в строке пропуск, удаляем строку

In [ ]:
# data = data.dropna(how='any', axis=0)

# data['lat'] = data['lat'].fillna(0, inplace=True)
# data['lng'] = data['lng'].fillna(0, inplace=True)
data = data.drop(columns=['lat', 'lng'])

Удалим дубликаты

In [ ]:
# не удалим, т.к. если удаляем то в последней ячейке - ошибка
# data = data.drop_duplicates()

Действия выше --- MAPE: 0.13401346221707813

#### Создание новых признаков (из признака tags)

In [ ]:
# функция перевода текста тегов в список
def tags_to_list(text):
    # list_len = len(text[2:-2].split('\', \''))
    tag_list = text[2:-2].split('\', \'')
    tag_list = [i.strip().lower() for i in tag_list]
    return tag_list

data['tags'] = data['tags'].apply(tags_to_list)

In [ ]:
tags = []
for tag_list in data['tags']:
    for tag in tag_list:
        tags.append(tag)
        
from collections import Counter
cnt = Counter(tags)
# cnt.most_common()

In [ ]:
for i in cnt.most_common():
    if i[1] > 2000 and 'night' not in i[0]:
        feature = i[0]
        data[feature] = data['tags'].apply(lambda x: 1 if feature in x else 0)

In [ ]:
# определение количества ночей
# определяем из списка тегов тег с количеством ночей
def nights_in_hotel(tags_list):
    
    for tag in tags_list:
        if 'night' in tag:
            for i in tag.split():
                if i.isdigit():
                    return i
    return np.NaN
# применяем функцию, получаем признак в строковом формате, внутри которого число (присутствуют пропуски)
data['nights_in_hotel'] = data['tags'].apply(nights_in_hotel)
# заполняем пропуски модой
data['nights_in_hotel'] = data['nights_in_hotel'].fillna(data['nights_in_hotel'].mode()[0])
# приводим к типу int
data['nights_in_hotel'] = data['nights_in_hotel'].astype('int')

# удаляем признак тега
data = data.drop(columns=['tags'])

Действия выше --- MAPE: 0.13286838323392136

#### <center> Преобразование признаков

Для нормализации признаков применим RobustScaler т.к не особо уделял вниманиче очистке от выбросов и нет уверенности, что признаки распределены нормально

In [ ]:
from sklearn import preprocessing

Применим преобразования над признаками - нормализацию

In [ ]:
data = data.reset_index()

In [ ]:
norm_cols_df = data[['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'nights_in_hotel']]

In [ ]:
# Копируем названия столбцов, которые теряются при использовании fit_transform()
col_names = list(norm_cols_df.columns)

# инициализируем нормализатор RobustScaler
r_scaler = preprocessing.RobustScaler()

# копируем исходный датасет
df_r = r_scaler.fit_transform(norm_cols_df)

df_r = pd.DataFrame(df_r, columns=col_names)

In [ ]:
data = data.drop(columns=['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'nights_in_hotel'])

data = pd.concat([data, df_r], axis=1)

#### Продолжение расчета

In [ ]:
data.nunique(dropna=False)

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических. 
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)